In [1]:
import neo4j

import pandas as pd

from IPython.display import display

### General imports, code from week 8 lab

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [6]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.code as node_code, labels(n) as labels
        order by n.code
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.code as node_code_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")
    

### Loading, formatting and cleaning routes data

In [7]:
routes_df = pd.read_csv('routes.csv')
routes_df.head(3)

,airline,airline ID,source airport,source airport id,destination apirport,destination airport id,codeshare,stops,equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2


In [8]:
routes_df.columns = [
    'airline',
    'airline_id',
    'source_airport',
    'source_airport_id',
    'destination_airport',
    'destination_airport_id',
    'codeshare',
    'stops',
    'equipment'
]

In [9]:
routes_df.head(3)

,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2


In [10]:
routes_df_dedupe = routes_df.drop_duplicates()

routes_df_clean = routes_df.dropna(subset=[
    'airline', 
    'airline_id', 
    'source_airport', 
    'source_airport_id', 
    'destination_airport', 
    'destination_airport_id', 
    'stops', 
    'equipment'
])

In [11]:
print(f'original shape:{routes_df.shape}')
print(f'deduped shape:{routes_df_dedupe.shape}')
print(f'non-null shape:{routes_df_clean.shape}')

routes_df = routes_df_clean

original shape:(67663, 9)
deduped shape:(67663, 9)
non-null shape:(67645, 9)


In [12]:
routes_df

,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [13]:
# airport.csv contains lat/lon location data for each airport (IATA is 3-letter airport code)

airports_df = pd.read_csv("airports.csv")
airports_df = airports_df[['IATA', 'Latitude', 'Longitude']].dropna()

# merge source coordinates
routes_df = routes_df.merge(
    airports_df, 
    left_on='source_airport', right_on='IATA'
).rename(columns={'Latitude': 'source_lat', 'Longitude': 'source_lon'}).drop('IATA', axis=1)

# merge destination coordinates
routes_df = routes_df.merge(
    airports_df, 
    left_on='destination_airport', right_on='IATA'
).rename(columns={'Latitude': 'dest_lat', 'Longitude': 'dest_lon'}).drop('IATA', axis=1)

routes_df.head()

,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,equipment,source_lat,source_lon,dest_lat,dest_lon
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2,43.449902,39.956600,55.606201,49.278702
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2,46.283298,48.006302,55.606201,49.278702
2,2B,410,CEK,2968,KZN,2990,NaN,0,CR2,55.305801,61.503300,55.606201,49.278702
3,2B,410,DME,4029,KZN,2990,NaN,0,CR2,55.408798,37.906300,55.606201,49.278702
4,S7,4329,DME,4029,KZN,2990,NaN,0,319,55.408798,37.906300,55.606201,49.278702


In [14]:
# Calculate geodesic distance between airports (from week 2 lab)

from geographiclib.geodesic import Geodesic

def my_calculate_distance(point_1, point_2):
    "Given two points in (latitude, longitude) format, calculate the distance between them in miles"

    geod = Geodesic.WGS84

    g = geod.Inverse(point_1[0], point_1[1], point_2[0], point_2[1])
    miles = g['s12'] / 1000 * 0.621371
    
    return miles


# use my_calculate_distance to create distance_miles column 

routes_df['distance_miles'] = routes_df.apply(
    lambda row: my_calculate_distance(
        (row['source_lat'], row['source_lon']),
        (row['dest_lat'], row['dest_lon'])
    ),
    axis=1
)

routes_df[['source_airport', 'destination_airport', 'distance_miles']].head()


,source_airport,destination_airport,distance_miles
0,AER,KZN,937.021056
1,ASF,KZN,646.811921
2,CEK,KZN,480.398224
3,DME,KZN,446.196699
4,DME,KZN,446.196699


In [15]:
# example: each airline's route between SFO and LAX (337 miles)

routes_df[(routes_df['source_airport'] == "SFO") & (routes_df['destination_airport'] == "LAX")]

,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,equipment,source_lat,source_lon,dest_lat,dest_lon,distance_miles
20129,AA,24,SFO,3469,LAX,3484,NaN,0,738,37.618999,-122.375,33.942501,-118.407997,337.442292
20130,AS,439,SFO,3469,LAX,3484,Y,0,738,37.618999,-122.375,33.942501,-118.407997,337.442292
20131,DL,2009,SFO,3469,LAX,3484,Y,0,E75,37.618999,-122.375,33.942501,-118.407997,337.442292
20132,UA,5209,SFO,3469,LAX,3484,NaN,0,319 320 752 738 753 739,37.618999,-122.375,33.942501,-118.407997,337.442292
20133,US,5265,SFO,3469,LAX,3484,NaN,0,738,37.618999,-122.375,33.942501,-118.407997,337.442292
20134,VX,5331,SFO,3469,LAX,3484,NaN,0,320 319,37.618999,-122.375,33.942501,-118.407997,337.442292
20135,WN,4547,SFO,3469,LAX,3484,NaN,0,733 73W 73C,37.618999,-122.375,33.942501,-118.407997,337.442292


In [16]:
# create inverse of distance (called 'weight') for louvain modularity

routes_df['weight'] = (1.0 / routes_df['distance_miles'])
routes_df.loc[routes_df['distance_miles'] == 0, 'weight'] = 1.0


In [17]:
routes_df[['source_airport', 'destination_airport', 'distance_miles', 'weight']].head()

,source_airport,destination_airport,distance_miles,weight
0,AER,KZN,937.021056,0.001067
1,ASF,KZN,646.811921,0.001546
2,CEK,KZN,480.398224,0.002082
3,DME,KZN,446.196699,0.002241
4,DME,KZN,446.196699,0.002241


In [18]:
routes_df['weight'].describe()

count    66916.000000
mean         0.002460
std          0.006904
min          0.000117
25%          0.000740
50%          0.001345
75%          0.002559
max          1.000000
Name: weight, dtype: float64

### Create airports, routes

In [19]:
airport_list = routes_df[['source_airport', 'source_lat', 'source_lon']].rename(
    columns={'source_airport': 'code', 'source_lat': 'lat', 'source_lon': 'lon'}
).drop_duplicates().to_dict('records')

airport_list[:5]

[{'code': 'AER', 'lat': 43.449902, 'lon': 39.9566},
 {'code': 'ASF', 'lat': 46.2832984924, 'lon': 48.0063018799},
 {'code': 'CEK', 'lat': 55.305801, 'lon': 61.5033},
 {'code': 'DME', 'lat': 55.40879821777344, 'lon': 37.90629959106445},
 {'code': 'EGO', 'lat': 50.643798828125, 'lon': 36.5900993347168}]

In [20]:
route_pairs = routes_df[['source_airport', 'destination_airport', 'distance_miles', 'weight']].rename(
    columns={'source_airport': 'from', 'destination_airport': 'to', 'distance_miles': 'distance'}
).to_dict('records')
route_pairs[:3]

[{'from': 'AER',
  'to': 'KZN',
  'distance': 937.0210555710688,
  'weight': 0.0010672118775287803},
 {'from': 'ASF',
  'to': 'KZN',
  'distance': 646.8119212199599,
  'weight': 0.001546044479381097},
 {'from': 'CEK',
  'to': 'KZN',
  'distance': 480.39822426138875,
  'weight': 0.002081606362174835}]

In [21]:
# take a list of airport codes and inserts them as Airport nodes in the Neo4j database

def insert_airports(airport_list):
    query = """
    UNWIND $airports AS ap
    MERGE (a:Airport {code: ap.code})
    SET a.lat = ap.lat,
        a.lon = ap.lon
    """
    session.run(query, airports=airport_list)

In [22]:
# takes a list of route dictionaries like:
# [
#     {'from': 'AER', 'to': 'KZN', 'distance_miles': 937.0210555710688},
#     {'from': 'ASF', 'to': 'KZN', 'distance_miles': 646.8119212199599}
# ]
# and creates FLIES_TO relationships between airports

def insert_routes(route_pairs):
    query = """
    UNWIND $routes AS route
    MATCH (a:Airport {code: route.from}),
          (b:Airport {code: route.to})
    MERGE (a)-[r:FLIES_TO]->(b)
    SET r.distance = route.distance,
        r.weight = route.weight
    """
    session.run(query, routes=route_pairs)


In [23]:
insert_airports(airport_list)
insert_routes(route_pairs)

### Create flightGraph projection

In [24]:
session.run("CALL gds.graph.drop('flightGraph', false) YIELD graphName;")

In [25]:
my_neo4j_wipe_out_database

<function __main__.my_neo4j_wipe_out_database()>

In [26]:
query = """
CALL gds.graph.project(
  'flightGraph',
  {
    Airport: {
      properties: ['lat', 'lon']
    }
  },
  {
    FLIES_TO: {
      type: 'FLIES_TO',
      orientation: 'NATURAL',
      properties: ['distance', 'weight']
    }
  }
)

"""
session.run(query)


### Algorithm 1: Shortest path

In [27]:
# Dijkstra's

source = "UAK" # (Narsarsuaq, Greenland)
target = "BQK" # (Brunswick, Georgia, USA)

query = f"""
MATCH (src:Airport {{code: '{source}'}}), (dst:Airport {{code: '{target}'}})
CALL gds.shortestPath.dijkstra.stream('flightGraph', {{
  sourceNode: id(src),
  targetNode: id(dst),
  relationshipWeightProperty: 'distance'
}})
YIELD totalCost, nodeIds
RETURN totalCost, 
       [nodeId IN nodeIds | gds.util.asNode(nodeId).code] AS path
"""
dijkstra_df = my_neo4j_run_query_pandas(query)
display(dijkstra_df)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 5, column: 15, offset: 156} for query: "\nMATCH (src:Airport {code: 'UAK'}), (dst:Airport {code: 'BQK'})\nCALL gds.shortestPath.dijkstra.stream('flightGraph', {\n  sourceNode: id(src),\n  targetNode: id(dst),\n  relationshipWeightProperty: 'distance'\n})\nYIELD totalCost, nodeIds\nRETURN totalCost, \n       [nodeId IN nodeIds | gds.util.asNode(nodeId).code] AS path\n"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 15, offset: 

,totalCost,path
0,4753.305518,"[UAK, GOH, KEF, JFK, ATL, BQK]"


In [28]:
# A Star (uses lat/lon for heuristic)

query = f"""
MATCH (src:Airport {{code: '{source}'}}), (dst:Airport {{code: '{target}'}})
CALL gds.shortestPath.astar.stream('flightGraph', {{
  sourceNode: id(src),
  targetNode: id(dst),
  relationshipWeightProperty: 'distance',
  latitudeProperty: 'lat',
  longitudeProperty: 'lon'
}})
YIELD totalCost, nodeIds
RETURN totalCost,
       [nodeId IN nodeIds | gds.util.asNode(nodeId).code] AS path
"""
astar_df = my_neo4j_run_query_pandas(query)
display(astar_df)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 5, column: 15, offset: 153} for query: "\nMATCH (src:Airport {code: 'UAK'}), (dst:Airport {code: 'BQK'})\nCALL gds.shortestPath.astar.stream('flightGraph', {\n  sourceNode: id(src),\n  targetNode: id(dst),\n  relationshipWeightProperty: 'distance',\n  latitudeProperty: 'lat',\n  longitudeProperty: 'lon'\n})\nYIELD totalCost, nodeIds\nRETURN totalCost,\n       [nodeId IN nodeIds | gds.util.asNode(nodeId).code] AS path\n"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated func

,totalCost,path
0,4733.669033,"[UAK, GOH, KEF, IAD, ATL, BQK]"
